In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from data_loader import load_bearing_data, compute_melspec, creadte_dataset_windows

In [2]:
DATA_DIR = '../data/raw/2nd_test'
files = sorted(os.listdir(DATA_DIR))

healthy_files = files[0:100]
failure_files = files[-5]

print(f'Loading {len(healthy_files)} data...')

all_windows = []
for filename in healthy_files:
	df = load_bearing_data(filename, DATA_DIR)
	melspec = compute_melspec(df)
	norm_mel = (melspec - (-80)) / (0 - (-80))
	windows = creadte_dataset_windows(norm_mel)
	all_windows.append(windows)

X_train = np.concatenate(all_windows, axis=0)
print(f'Traing data shape: {X_train.shape}')

df_failure = load_bearing_data(failure_files, DATA_DIR)
mel_fail = compute_melspec(df_failure)
norm_fail = (mel_fail - (-80)) / (0 - (-80))
X_test_anomaly = creadte_dataset_windows(norm_fail)

print(f'Anomaly shape: {X_test_anomaly.shape}')

Loading 100 data...
Traing data shape: (400, 128, 64, 1)
Anomaly shape: (4, 128, 64, 1)


In [4]:
def build_autoencoder(input_shape):
	model = models.Sequential()
	model.add(layers.Input(shape=input_shape))
	
	model.add(layers.Conv2D(32, (3,3), activation='relu', padding='same'))
	model.add(layers.MaxPooling2D((2,2), padding='same'))
	model.add(layers.Conv2D(16, (3,3), activation='relu', padding='same'))
	model.add(layers.MaxPooling2D((2,2), padding='same'))

	model.add(layers.Conv2D(16, (3,3), activation='relu', padding='same'))
	model.add(layers.UpSampling2D((2,2)))
	model.add(layers.Conv2D(32, (3,3), activation='relu', padding='same'))
	model.add(layers.UpSampling2D((2,2)))

	model.add(layers.Conv2D(1, (3,3), activation='sigmoid', padding='same'))

	return model

input_shape = (128, 64, 1)
model = build_autoencoder(input_shape)

model.compile(optimizer='adam', loss='mse')

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 128, 64, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 64, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 64, 32, 16)     │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 32, 16, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 32, 16, 16)     │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 64, 32, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 64, 32, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 128, 64, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 128, 64, 1)     │           289 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,193 (47.63 KB)

 Trainable params: 12,193 (47.63 KB)

 Non-trainable params: 0 (0.00 B)